🚀 머신러닝 실습 : 고객 구매 데이터로 성별 예측 모델링 (분류 문제)
- 주어진 데이터는 백화점 고객의 1년 간 구매 데이터입니다.
- 고객 3,500명에 대한 학습용 데이터(y.csv, X.csv)를 이용하여 성별예측 모형을 만들어보세요.
- 모델의 성능은 자유롭게 측정해봅니다!

[실습 프로세스]
1. 데이터 불러오기
2. 데이터 탐색
3. 데이터 전처리
4. 학습/테스트 데이터 분리
5. 모델 선택 및 학습
6. 예측 및 평가

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
X = pd.read_csv('./data/X.csv', encoding='euc-kr')
y = pd.read_csv('./data/y.csv', encoding='euc-kr')

1. 데이터 불러오기
- 데이터를 가져와서 과정을 준비합시다.
- 인코딩 방식은 'euc-kr' 을 활용하세요.
- 데이터 출처 : 한국데이터산업진흥원 빅데이터분석기사 실기 공개 예시 문항
- 독립 변수 데이터셋 : ./data/X.csv
- 종속 변수 데이터셋 : ./data/y.csv

데이터 파일을 불러옵니다. 보통 CSV 파일을 pandas로 읽어옵니다.

In [3]:
X.head()

,cust_id,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,0,68282840,11264000,6860000.0,기타,강남점,19,3.894737,0.527027,17
1,1,2136000,2136000,300000.0,스포츠,잠실점,2,1.500000,0.000000,1
2,2,3197000,1639000,NaN,남성 캐주얼,관악점,2,2.000000,0.000000,1
3,3,16077620,4935000,NaN,기타,광주점,18,2.444444,0.318182,16
4,4,29050000,24000000,NaN,보석,본 점,2,1.500000,0.000000,85


In [4]:
y.head()

,cust_id,gender
0,0,0
1,1,0
2,2,1
3,3,1
4,4,0


2. 데이터 탐색하기
- 데이터를 이해할 수 있도록 탐색과정을 수행해봅시다.

데이터의 상위 몇 개 행을 출력하여 전체 구조를 미리 확인합니다.

데이터의 요약 정보나 통계 정보를 출력해 변수들의 유형과 분포를 확인합니다.

데이터의 요약 정보나 통계 정보를 출력해 변수들의 유형과 분포를 확인합니다.

In [5]:
print(X.head())
print(y.head())

   cust_id      총구매액     최대구매액       환불금액   주구매상품 주구매지점  내점일수   내점당구매건수  \
0        0  68282840  11264000  6860000.0      기타   강남점    19  3.894737   
1        1   2136000   2136000   300000.0     스포츠   잠실점     2  1.500000   
2        2   3197000   1639000        NaN  남성 캐주얼   관악점     2  2.000000   
3        3  16077620   4935000        NaN      기타   광주점    18  2.444444   
4        4  29050000  24000000        NaN      보석  본  점     2  1.500000   

     주말방문비율  구매주기  
0  0.527027    17  
1  0.000000     1  
2  0.000000     1  
3  0.318182    16  
4  0.000000    85  
   cust_id  gender
0        0       0
1        1       0
2        2       1
3        3       1
4        4       0


In [6]:
print(X.info())
print(y.info())

<class 'pandas.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cust_id  3500 non-null   int64  
 1   총구매액     3500 non-null   int64  
 2   최대구매액    3500 non-null   int64  
 3   환불금액     1205 non-null   float64
 4   주구매상품    3500 non-null   str    
 5   주구매지점    3500 non-null   str    
 6   내점일수     3500 non-null   int64  
 7   내점당구매건수  3500 non-null   float64
 8   주말방문비율   3500 non-null   float64
 9   구매주기     3500 non-null   int64  
dtypes: float64(3), int64(5), str(2)
memory usage: 273.6 KB
None
<class 'pandas.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   cust_id  3500 non-null   int64
 1   gender   3500 non-null   int64
dtypes: int64(2)
memory usage: 54.8 KB
None


In [7]:
print(X.describe())
print(y.value_counts())

           cust_id          총구매액         최대구매액          환불금액         내점일수  \
count  3500.000000  3.500000e+03  3.500000e+03  1.205000e+03  3500.000000   
mean   1749.500000  9.191925e+07  1.966424e+07  2.407822e+07    19.253714   
std    1010.507298  1.635065e+08  3.199235e+07  4.746453e+07    27.174942   
min       0.000000 -5.242152e+07 -2.992000e+06  5.600000e+03     1.000000   
25%     874.750000  4.747050e+06  2.875000e+06  2.259000e+06     2.000000   
50%    1749.500000  2.822270e+07  9.837000e+06  7.392000e+06     8.000000   
75%    2624.250000  1.065079e+08  2.296250e+07  2.412000e+07    25.000000   
max    3499.000000  2.323180e+09  7.066290e+08  5.637530e+08   285.000000   

           내점당구매건수       주말방문비율         구매주기  
count  3500.000000  3500.000000  3500.000000  
mean      2.834963     0.307246    20.958286  
std       1.912368     0.289752    24.748682  
min       1.000000     0.000000     0.000000  
25%       1.666667     0.027291     4.000000  
50%       2.333333     0

In [8]:
print(X.isnull().sum())

cust_id       0
총구매액          0
최대구매액         0
환불금액       2295
주구매상품         0
주구매지점         0
내점일수          0
내점당구매건수       0
주말방문비율        0
구매주기          0
dtype: int64


In [9]:
print(y['gender'].value_counts(normalize=True))

gender
0    0.624
1    0.376
Name: proportion, dtype: float64


3. 데이터 전처리
- 전처리 과정을 통해서 머신러닝에 사용할 수 있는 형태의 데이터 준비
필요한 라이브러리를 불러옵니다.

- 인코딩 : LabelEncoder
- 데이터 표준화 : StandardScaler

- 단순히 1부터의 숫자를 부여한 'cust_id'를 수치형 변수로 받아들이면, 결과가 왜곡될 수 있으니 컬럼을 제거합니다.

- 데이터에 결측치가 있는지 확인해보세요

- 결측치에 0으로 채워 넣어 모델 학습에 지장이 없도록 합니다.

문자형 범주 데이터를 숫자로 바꾸기 위한 인코딩을 수행합니다.

각 데이터에 표준화를 적용하여 데이터의 스케일(크기 차이)을 맞춰줍니다.

- 평균을 0, 표준편차를 1로 맞춰서 → 데이터가 정규 분포 형태로 변환되도록 하세요

In [10]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np

In [11]:
X = X.drop(columns=['cust_id'])

In [12]:
print("결측치 개수 확인")
print(X.isnull().sum())

결측치 개수 확인
총구매액          0
최대구매액         0
환불금액       2295
주구매상품         0
주구매지점         0
내점일수          0
내점당구매건수       0
주말방문비율        0
구매주기          0
dtype: int64


In [13]:
X = X.fillna(0)

In [14]:
print(X.dtypes)

총구매액         int64
최대구매액        int64
환불금액       float64
주구매상품          str
주구매지점          str
내점일수         int64
내점당구매건수    float64
주말방문비율     float64
구매주기         int64
dtype: object


In [15]:
categorical_cols = X.select_dtypes(include=['object']).columns

In [16]:
le = LabelEncoder()

for col in categorical_cols:
    X[col] = le.fit_transform(X[col])

In [17]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("전처리 완료")

전처리 완료


5-1. 모델링 - LogisticRegression
본격적으로 모델을 선언하고 학습시킵니다.
필요한 라이브러리를 불러옵니다.

모델을 선언하여 객체화시킵니다.

모델을 학습 데이터에 맞춰 학습시킵니다.


In [20]:
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [21]:
# ----------------------------
# 0) (중요) y를 1차원 라벨로 정리
# ----------------------------
# y가 pandas Series면 그대로 OK
# y가 DataFrame(열이 1개 이상)이거나 numpy 2차원이면 1차원으로 펴야 함

# 케이스 A: y가 pandas DataFrame이고 라벨 컬럼이 'target'인 경우
# y = y['target']

# 케이스 B: y가 pandas DataFrame인데 라벨이 첫번째 열인 경우
# y = y.iloc[:, 0]

# 케이스 C: y가 numpy 배열인데 2차원일 수 있는 경우 (가장 안전)
y = np.array(y).ravel()  # (n,) 형태로 강제

print("X type/shape:", type(X), getattr(X, "shape", None))
print("y type/shape:", type(y), y.shape)
print("y unique (앞 20개):", np.unique(y)[:20])

# ----------------------------
# 1) train/test split (stratify는 y가 1차원일 때만 정상 동작)
# ----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


X type/shape: <class 'pandas.DataFrame'> (3500, 9)
y type/shape: <class 'numpy.ndarray'> (7000,)
y unique (앞 20개): [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


ValueError: Found input variables with inconsistent numbers of samples: [3500, 7000]

In [22]:
# ----------------------------
# 2) 스케일링 (fit은 train에만!)
# ----------------------------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

print("X_train_scaled shape:", X_train_scaled.shape)
print("X_test_scaled shape:", X_test_scaled.shape)

# ----------------------------
# 3) LogisticRegression 모델링(학습)
# ----------------------------
log_model = LogisticRegression(
    max_iter=1000,
    class_weight="balanced",   # 너가 balanced 쓰는 흐름이면 유지, 아니면 None
    random_state=42
)

log_model.fit(X_train_scaled, y_train)

print("LogisticRegression 학습 완료")

NameError: name 'X_train' is not defined

In [ ]:

from sklearn.linear_model import LogisticRegression  # 로지스틱 회귀(분류) 모델 불러오기

# 1) 모델 선언(객체화)
# - max_iter: 반복 학습 횟수(수렴 안 하면 에러/경고 나서 넉넉히)
# - class_weight: 불균형 데이터면 'balanced'가 도움될 수 있음(필요 없으면 제거)
# - random_state: 결과 재현성
log_model = LogisticRegression(
    max_iter=1000,
    class_weight=None,   # 불균형 보정 필요하면 'balanced'로 바꾸세요
    random_state=42
)

# 2) 모델 학습
# ⚠ 반드시 스케일링된 X_train_scaled를 사용해야 함
log_model.fit(X_train_scaled, y_train)

# (선택) 학습이 잘 됐는지 간단 확인
print("학습 완료. coef shape:", log_model.coef_.shape)

학습 완료. coef shape: (1, 9)


# 6-1. 예측 성능 확인해보기 - LogisticRegression
- 학습된 모델로 테스트 데이터에 대한 예측을 수행합니다.

- 학습시킨 모델의 성능을 알아봅니다
- 각 평가지표로 모델의 성능을 수치화하여 확인합니다.
- 필요한 라이브러리를 import 하고 성능을 확인해보세요 (정확도, 정밀도, 재현율, f1, confusion_matrix)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
# 2. 정확도
acc = accuracy_score(y_test, y_pred)

# 3. 정밀도
prec = precision_score(y_test, y_pred)

# 4. 재현율
rec = recall_score(y_test, y_pred)

# 5. F1 Score
f1 = f1_score(y_test, y_pred)

# 6. 혼동행렬
cm = confusion_matrix(y_test, y_pred)

print("Accuracy:", acc)
print("Precision:", prec)
print("Recall:", rec)
print("F1 Score:", f1)
print("Confusion Matrix:\n", cm)

Accuracy: 0.6328571428571429
Precision: 0.5454545454545454
Recall: 0.13688212927756654
F1 Score: 0.2188449848024316
Confusion Matrix:
 [[407  30]
 [227  36]]


In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.64      0.93      0.76       437
           1       0.55      0.14      0.22       263

    accuracy                           0.63       700
   macro avg       0.59      0.53      0.49       700
weighted avg       0.61      0.63      0.56       700



# 5-2. 모델링 - DecisionTreeClassifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier(
    max_depth=5,        # 과적합 방지
    random_state=42
)

dt_model.fit(X_train, y_train)

,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.",'gini'
,"splitter splitter: {""best"", ""random""}, default=""best""The strategy used to choose the split at each node. Supportedstrategies are ""best"" to choose the best split and ""random"" to choosethe best random split.",'best'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",5
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: int, float or {""sqrt"", ""log2""}, default=NoneThe number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... note:: The search for a split does not stop until at least one valid partition of the node samples is found, even if it requires to effectively inspect more than ``max_features`` features.",None
,"random_state random_state: int, RandomState instance or None, default=NoneControls the randomness of the estimator. The features are alwaysrandomly permuted at each split, even if ``splitter`` is set to``""best""``. When ``max_features < n_features``, the algorithm willselect ``max_features`` at random at each split before finding the bestsplit among them. But the best found split may vary across differentruns, even if ``max_features=n_features``. That is the case, if theimprovement of the criterion is identical for several splits and onesplit has to be selected at random. To obtain a deterministic behaviourduring fitting, ``random_state`` has to be fixed to an integer.See :term:`Glossary ` for details.",42
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow a tree with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current no

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

y_pred_dt = dt_model.predict(X_test)


acc_dt = accuracy_score(y_test, y_pred_dt)
prec_dt = precision_score(y_test, y_pred_dt)
rec_dt = recall_score(y_test, y_pred_dt)
f1_dt = f1_score(y_test, y_pred_dt)
cm_dt = confusion_matrix(y_test, y_pred_dt)

print("=== Decision Tree 성능 ===")
print("Accuracy:", acc_dt)
print("Precision:", prec_dt)
print("Recall:", rec_dt)
print("F1 Score:", f1_dt)
print("Confusion Matrix:\n", cm_dt)

=== Decision Tree 성능 ===
Accuracy: 0.63
Precision: 0.515625
Recall: 0.2509505703422053
F1 Score: 0.3375959079283887
Confusion Matrix:
 [[375  62]
 [197  66]]


In [ ]:
print(classification_report(y_test, y_pred_dt))

              precision    recall  f1-score   support

           0       0.66      0.86      0.74       437
           1       0.52      0.25      0.34       263

    accuracy                           0.63       700
   macro avg       0.59      0.55      0.54       700
weighted avg       0.60      0.63      0.59       700



# 5-3. 모델링 – RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_model = RandomForestClassifier(
    n_estimators=300,      # 트리 개수 (많을수록 안정적)
    max_depth=None,        # 깊이 제한 없음 (기본)
    min_samples_split=5,   # 과적합 방지
    min_samples_leaf=3,    # 리프 최소 샘플 수
    class_weight='balanced',
    random_state=42,
    n_jobs=-1              # CPU 전부 사용 (속도 향상)
)

In [ ]:
rf_model.fit(X_train, y_train)

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",300
,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.Note: This parameter is tree-specific.",'gini'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",5
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",3
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=""sqrt""The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to `""sqrt""`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",'sqrt'
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft child, and ``N_t_R`` is the number of samples in the right child.``N``, ``N_t``, ``N_t_R`` and ``N_t_L`` all refer to the weighted sum,if ``sample_weight`` is passed... versionadded:: 0.19",0.0
,"bootstrap bootstrap: bool, default=TrueWhether bootstrap samples are used when building trees. If False, thewhole dataset is used to build each tree.",True
,"oob_score oob_score: bool or callable, default=FalseWhether to use out-of-bag samples to estimate the generalization score.By default, :func:`~sklearn.metrics.accuracy_score` is used.Provide a callable with signature `metric

# 6-3. 예측 및 성능 평가

In [ ]:
y_pred_rf = rf_model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

acc_rf = accuracy_score(y_test, y_pred_rf)
prec_rf = precision_score(y_test, y_pred_rf)
rec_rf = recall_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)
cm_rf = confusion_matrix(y_test, y_pred_rf)

print("=== RandomForest 성능 ===")
print("Accuracy:", acc_rf)
print("Precision:", prec_rf)
print("Recall:", rec_rf)
print("F1 Score:", f1_rf)
print("Confusion Matrix:\n", cm_rf)

=== RandomForest 성능 ===
Accuracy: 0.6357142857142857
Precision: 0.5181818181818182
Recall: 0.43346007604562736
F1 Score: 0.4720496894409938
Confusion Matrix:
 [[331 106]
 [149 114]]


In [ ]:
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.69      0.76      0.72       437
           1       0.52      0.43      0.47       263

    accuracy                           0.64       700
   macro avg       0.60      0.60      0.60       700
weighted avg       0.63      0.64      0.63       700



# 5-4. 모델링 – XGBoost

In [ ]:
from xgboost import XGBClassifier

In [ ]:
xgb_model = XGBClassifier(
    n_estimators=300,        # 트리 개수
    max_depth=4,             # 트리 깊이
    learning_rate=0.05,      # 학습률 (작을수록 안정적)
    subsample=0.8,           # 데이터 샘플링
    colsample_bytree=0.8,    # 변수 샘플링
    random_state=42,
    scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum(),  # 불균형 보정
    eval_metric='logloss',
    use_label_encoder=False
)

In [ ]:
xgb_model.fit(X_train, y_train)

,"objective objective: typing.Union[str, xgboost.sklearn._SklObjWProto, typing.Callable[[typing.Any, typing.Any], typing.Tuple[numpy.ndarray, numpy.ndarray]], NoneType]Specify the learning task and the corresponding learning objective or a customobjective function to be used.For custom objective, see :doc:`/tutorials/custom_metric_obj` and:ref:`custom-obj-metric` for more information, along with the end note forfunction signatures.",'binary:logistic'
,"base_score base_score: typing.Union[float, typing.List[float], NoneType]The initial prediction score of all instances, global bias.",None
,booster,None
,"callbacks callbacks: typing.Optional[typing.List[xgboost.callback.TrainingCallback]]List of callback functions that are applied at end of each iteration.It is possible to use predefined callbacks by using:ref:`Callback API `... note:: States in callback are not preserved during training, which means callback objects can not be reused for multiple training sessions without reinitialization or deepcopy... code-block:: python for params in parameters_grid: # be sure to (re)initialize the callbacks before each run callbacks = [xgb.callback.LearningRateScheduler(custom_rates)] reg = xgboost.XGBRegressor(**params, callbacks=callbacks) reg.fit(X, y)",None
,colsample_bylevel colsample_bylevel: typing.Optional[float]Subsample ratio of columns for each level.,None
,colsample_bynode colsample_bynode: typing.Optional[float]Subsample ratio of columns for each split.,None
,colsample_bytree colsample_bytree: typing.Optional[float]Subsample ratio of columns when constructing each tree.,0.8
,"device device: typing.Optional[str].. versionadded:: 2.0.0Device ordinal, available options are `cpu`, `cuda`, and `gpu`.",None
,"early_stopping_rounds early_stopping_rounds: typing.Optional[int].. versionadded:: 1.6.0- Activates early stopping. Validation metric needs to improve at least once in every **early_stopping_rounds** round(s) to continue training. Requires at least one item in **eval_set** in :py:meth:`fit`.- If early stopping occurs, the model will have two additional attributes: :py:attr:`best_score` and :py:attr:`best_iteration`. These are used by the :py:meth:`predict` and :py:meth:`apply` methods to determine the optimal number of trees during inference. If users want to access the full model (including trees built after early stopping), they can specify the `iteration_range` in these inference methods. In addition, other utilities like model plotting can also use the entire model.- If you prefer to discard the trees after `best_iteration`, consider using the callback function :py:class:`xgboost.callback.EarlyStopping`.- If there's more than one item in **eval_set**, the last entry will be used for early stopping. If there's more than one metric in **eval_metric**, the last metric will be used for early stopping.",None
,enable_categorical enable_categorical: boolSee the same parameter of :py:class:`DMatrix` for details.,False
,"eval_metric eval_metric: typing.Union[str, typing.List[typing.Union[str, typing.Callable]], typing.Callable, NoneType].. versionadded:: 1.6.0Metric used for monitoring the training result and early stopping. It can be astring or list of strings as names of predefined metric in XGBoost (See:doc:`/parameter`), one of the metrics in :py:mod:`sklearn.metrics`, or anyother user defined metric that looks like `sklearn.metrics`.If custom objective is also provided, then custom metric should implement thecorresponding reverse link function.Unlike the `scoring` parameter commonly used in scikit-learn, when a callableobject is provided, it's assumed to be a cost function and by default XGBoostwill minimize the result during early stopping.For advanced usage on Early stopping like directly choosing to maximize insteadof minimize, see :py:obj:`xgboost.callback.EarlyStopping`.See :doc:`/tutorials/custom_metric_obj` and :ref:`custom-obj-metric` for moreinformation... code-block:: python from sklearn.datasets import load_diabetes f

In [ ]:
y_pred_xgb = xgb_model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

acc_xgb = accuracy_score(y_test, y_pred_xgb)
prec_xgb = precision_score(y_test, y_pred_xgb)
rec_xgb = recall_score(y_test, y_pred_xgb)
f1_xgb = f1_score(y_test, y_pred_xgb)
cm_xgb = confusion_matrix(y_test, y_pred_xgb)

print("=== XGBoost 성능 ===")
print("Accuracy:", acc_xgb)
print("Precision:", prec_xgb)
print("Recall:", rec_xgb)
print("F1 Score:", f1_xgb)
print("Confusion Matrix:\n", cm_xgb)

=== XGBoost 성능 ===
Accuracy: 0.5785714285714286
Precision: 0.44666666666666666
Recall: 0.5095057034220533
F1 Score: 0.47602131438721135
Confusion Matrix:
 [[271 166]
 [129 134]]


In [ ]:
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.68      0.62      0.65       437
           1       0.45      0.51      0.48       263

    accuracy                           0.58       700
   macro avg       0.56      0.56      0.56       700
weighted avg       0.59      0.58      0.58       700



# 7. 위 4가지 모델의 학습 & 예측 & 평가 결과를 확인하고 최고 성능을 내는 모델을 찾아봅시다!
어떤 모델이 가장 성능이 좋은가요 ?

In [ ]:
results = pd.DataFrame({
    'Model': ['Logistic', 'DecisionTree', 'RandomForest', 'XGBoost'],
    'Accuracy': [acc, acc_dt, acc_rf, acc_xgb],
    'F1': [f1, f1_dt, f1_rf, f1_xgb]
})

print(results.sort_values(by='Accuracy', ascending=False))

          Model  Accuracy        F1
2  RandomForest  0.635714  0.472050
0      Logistic  0.632857  0.218845
1  DecisionTree  0.630000  0.337596
3       XGBoost  0.578571  0.476021


In [ ]:
# 정밀도랑 정확도를 다시 비교 RandomForest, XGBoost 

In [ ]:
from sklearn.metrics import accuracy_score, precision_score

# RandomForest
acc_rf = accuracy_score(y_test, y_pred_rf)
prec_rf = precision_score(y_test, y_pred_rf)

# XGBoost
acc_xgb = accuracy_score(y_test, y_pred_xgb)
prec_xgb = precision_score(y_test, y_pred_xgb)

print("RandomForest - Accuracy:", acc_rf)
print("RandomForest - Precision:", prec_rf)

print("XGBoost - Accuracy:", acc_xgb)
print("XGBoost - Precision:", prec_xgb)

RandomForest - Accuracy: 0.6357142857142857
RandomForest - Precision: 0.5181818181818182
XGBoost - Accuracy: 0.5785714285714286
XGBoost - Precision: 0.44666666666666666


In [ ]:
import pandas as pd

comparison = pd.DataFrame({
    'Model': ['RandomForest', 'XGBoost'],
    'Accuracy': [acc_rf, acc_xgb],
    'Precision': [prec_rf, prec_xgb]
})

print(comparison.sort_values(by='Accuracy', ascending=False))

          Model  Accuracy  Precision
0  RandomForest  0.635714   0.518182
1       XGBoost  0.578571   0.446667
